In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os
import cv2,json,threading
from tqdm import tqdm
ignore_files = ['analysis.ipynb', '.ipynb_checkpoints','data_creation.py']

In [2]:
!ls

analytics.ipynb		      IN_youtube_trending_data.csv
BR_youtube_trending_data.csv  JPvideos.csv
CAvideos.csv		      JP_youtube_trending_data.csv
CA_youtube_trending_data.csv  KRvideos.csv
data_creation.py	      KR_youtube_trending_data.csv
DEvideos.csv		      MXvideos.csv
DE_youtube_trending_data.csv  MX_youtube_trending_data.csv
FRvideos.csv		      RUvideos.csv
FR_youtube_trending_data.csv  RU_youtube_trending_data.csv
GBvideos.csv		      USvideos.csv
GB_youtube_trending_data.csv  US_youtube_trending_data.csv
INvideos.csv


In [3]:
data1 = pd.read_csv('./US_youtube_trending_data.csv')

In [4]:
len(data1)

100191

In [5]:
data2 = pd.read_csv('./USvideos.csv')

In [6]:
len(data2)

40949

In [7]:
files = list(os.listdir('./'))
files.remove('US_youtube_trending_data.csv')
files.remove('USvideos.csv')
files.remove('.ipynb_checkpoints')
files.remove('analytics.ipynb')
files.remove('data_creation.py')
files.remove('.virtual_documents')

In [8]:
for file in tqdm(files):
    if len(file.split('_')) > 1:
        data1 = data1.append(pd.read_csv(f'./{file}'))
    else:
        data2 = data2.append(pd.read_csv(f'./{file}',encoding='latin-1'))

100%|███████████████████████| 19/19 [00:18<00:00,  1.05it/s]


In [9]:
len(data1)

1092008

In [10]:
len(data2)

375942

In [11]:
data1.rename(columns={'publishedAt':'publish_time','channelTitle':'channel_title','categoryId':'category_id','view_count':'views'},inplace=True)

In [12]:
data2.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [13]:
data = data1.append(data2)

In [14]:
len(data)

1467950

In [15]:
data.head()

,video_id,title,publish_time,channelId,channel_title,category_id,trending_date,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,video_error_or_removed
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,NaN
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare...",NaN
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...,NaN
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...,NaN
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...,NaN


In [16]:
data.isna().sum()

video_id                        0
title                           0
publish_time                    0
channelId                  375942
channel_title                   1
category_id                     0
trending_date                   0
tags                            0
views                           0
likes                           0
dislikes                        0
comment_count                   0
thumbnail_link                  0
comments_disabled               0
ratings_disabled                0
description                 44945
video_error_or_removed    1092008
dtype: int64

In [21]:
data.drop(['channelId','description','video_error_or_removed'],axis=1,inplace=True)

In [22]:
data.isna().sum()

video_id             0
title                0
publish_time         0
channel_title        1
category_id          0
trending_date        0
tags                 0
views                0
likes                0
dislikes             0
comment_count        0
thumbnail_link       0
comments_disabled    0
ratings_disabled     0
dtype: int64

In [24]:
data.dropna(inplace=True)

In [25]:
data.isna().sum()

video_id             0
title                0
publish_time         0
channel_title        0
category_id          0
trending_date        0
tags                 0
views                0
likes                0
dislikes             0
comment_count        0
thumbnail_link       0
comments_disabled    0
ratings_disabled     0
dtype: int64